# Simulate AWS DMS output

In my thesis, the input files are pretended to be written by AWS DMS. Instead of actually creating a relational database and extracting it using AWS DMS, I have chosen to directly create the output files.

In [1]:
import pandas as pd
import random
import os
from helpers.paths import PathMerger
from faker import Faker
from datetime import datetime, timedelta, date, timezone
from dateutil import tz
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

fake = Faker()

## Functions

In [2]:
def generate_customers(first_registration:datetime):
    customers = []
    
    # Memorized value for the creation time of the previous customer.
    created_prev = None
    
    # A single hard-coded customer for reference with an id of 0
    customer = {
        'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'id': 0, 
        'username': 'janisourander@kamk.fi',
        # 'password': fake.md5(),
        'created': datetime(1970, 1, 15, 10, 0, 0),
        'modified': datetime(1970, 2, 20, 12, 34, 56),
    }
    
    customers.append(customer)

    for i in range(9):
        
        if created_prev is None:
            # The first ever customer was created at 13:37, 15th of Jan 2020.
            created = first_registration
        else:
            # Each other customer were created n second later than the previous.
            created = created_prev + timedelta(seconds=random.randrange(1, 3600))
        
        customer = {
            'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'id': len(customers), 
            'username': fake.unique.ascii_email(),
             # 'password': fake.md5(),
            'created': created,
            'modified': created + timedelta(seconds=random.randrange(0, 3600 * 24 * 365)),
        }
        
        customers.append(customer)
        
        # Set current created datetime as previous
        created_prev = created

    return pd.DataFrame.from_records(customers)
    
    
def generate_device_models():
    devices = [
        {
            'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'id': 0, 
            'release_date': date(2010, 5, 15),
            'name': 'Super Gadget 100',
            'color': 'Red',
            'description': 'lorem ipsum',
            'created': datetime(2010, 3, 21, 12, 0, 1),
            'modified': datetime(2010, 3, 21, 12, 0, 1),
        },
        {
            'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'id': 1, 
            'release_date': date(2010, 5, 15),
            'name': 'Super Gadget 100',
            'color': 'Black',
            'description': 'lorem ipsum',
            'created': datetime(2010, 3, 21, 12, 0, 2),
            'modified': datetime(2010, 3, 21, 12, 0, 2),
        },
        {
            'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'id': 2, 
            'release_date': date(2010, 11, 1),
            'name': 'Super Gadget 100',
            'color': 'Pink',
            'description': 'lorem ipsum',
            'created': datetime(2010, 8, 5, 7, 0, 0),
            'modified': datetime(2010, 8, 5, 7, 0, 0),
        },
        {
            'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'id': 3, 
            'release_date': date(2018, 5, 13),
            'name': 'Super Gadget 200',
            'color': 'White',
            'description': 'lorem ipsum',
            'created': datetime(2018, 3, 20, 12, 1, 1),
            'modified': datetime(2018, 3, 20, 12, 1, 1),
        },
    ]
    
    return pd.DataFrame.from_records(devices)


def generate_devices(customers: pd.DataFrame, devices: pd.DataFrame):

    # List of unique model numbers
    unique_device_models = list(devices.id.unique())
    
    # Container
    devices = []
    i = 0

    for (_, cust) in customers.iterrows():
        
        # Fetch the creation date of user and add some
        created = cust.created + timedelta(seconds=random.randrange(1, 3600))
        
        # Create 1 or 2 devices per user by random
        for x in range(random.randrange(1, 3)):
            device = {
                'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'id': len(devices), 
                'customer_fk': cust.id,
                'model_fk': random.choice(unique_device_models),
                'serial_number': fake.ssn(),
                'created': created,
                'modified': created + timedelta(seconds=random.randrange(1, 3600)),
            }

            devices.append(device)
    
    return pd.DataFrame.from_records(devices)

        
def generate_customer_details(customers: pd.DataFrame):
    
    # Container
    details = []
    
    for (_, cust) in customers.iterrows():
        detail = {
            'dms_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'id': len(details),
            'customer_fk': cust.id,
            'birthday': fake.date_of_birth(),
            'language': fake.language_code(),
            'street_address': fake.street_address(),
            'postal_code': fake.postalcode(),
            'city': fake.city(),
            'country': fake.country_code(),
            'phone_number': fake.phone_number(),
            'created': cust.created,
            'modified': cust.created,
        }
        
        details.append(detail)
        
    return pd.DataFrame.from_records(details)

## Create Directories

In [3]:
# Table paths
table_paths = {}

# Map of combinations to create: (db, table, data_generator_function)
tables = [
    ('customers', 'customers'), 
    ('customers', 'customer_details'),
    ('devices', 'device_models'),
    ('devices', 'devices')
]

for db, table in tables:
    dt = f'{db}.{table}'
    sp = PathMerger(db, table).staging
    dp = os.path.join(sp, 'LOAD00000001.parquet')
    
    # Create dirs
    if not os.path.exists(sp):
        os.makedirs(sp)
        
    table_paths[dt] = dp

## Generate Datasets

### Customers

In [4]:
# Generate
df_c = generate_customers(first_registration=datetime(2020, 1, 15, 13, 37, 0))

# Write
df_c.to_parquet(table_paths['customers.customers'])

# Show
df_c

,dms_timestamp,id,username,created,modified
0,2021-08-05 09:30:47,0,janisourander@kamk.fi,1970-01-15 10:00:00,1970-02-20 12:34:56
1,2021-08-05 09:30:47,1,rscott@gmail.com,2020-01-15 13:37:00,2020-02-26 22:00:52
2,2021-08-05 09:30:47,2,wfletcher@yahoo.com,2020-01-15 14:29:08,2020-02-21 22:57:29
3,2021-08-05 09:30:47,3,vhoward@gmail.com,2020-01-15 14:41:13,2020-10-27 17:35:24
4,2021-08-05 09:30:47,4,petersonmelissa@yahoo.com,2020-01-15 14:58:59,2020-02-15 13:58:09
5,2021-08-05 09:30:47,5,andrew17@lopez.info,2020-01-15 15:36:56,2020-03-09 06:12:44
6,2021-08-05 09:30:47,6,nkennedy@carson.biz,2020-01-15 15:55:33,2020-01-28 10:53:03
7,2021-08-05 09:30:47,7,james44@hotmail.com,2020-01-15 16:17:08,2020-03-24 08:49:00
8,2021-08-05 09:30:47,8,eibarra@yahoo.com,2020-01-15 17:12:11,2020-11-02 20:20:21
9,2021-08-05 09:30:47,9,ctorres@yahoo.com,2020-01-15 17:36:47,2020-06-04 20:50:32


### Device Models

In [5]:
# Generate
df_dm = generate_device_models()

# Write
df_dm.to_parquet(table_paths['devices.device_models'])

# Show
df_dm

,dms_timestamp,id,release_date,name,color,description,created,modified
0,2021-08-05 09:30:47,0,2010-05-15,Super Gadget 100,Red,lorem ipsum,2010-03-21 12:00:01,2010-03-21 12:00:01
1,2021-08-05 09:30:47,1,2010-05-15,Super Gadget 100,Black,lorem ipsum,2010-03-21 12:00:02,2010-03-21 12:00:02
2,2021-08-05 09:30:47,2,2010-11-01,Super Gadget 100,Pink,lorem ipsum,2010-08-05 07:00:00,2010-08-05 07:00:00
3,2021-08-05 09:30:47,3,2018-05-13,Super Gadget 200,White,lorem ipsum,2018-03-20 12:01:01,2018-03-20 12:01:01


### Devices

In [6]:
# Generate devices based on customers and device models
df_d = generate_devices(df_c, df_dm)

# Write
df_d.to_parquet(table_paths['devices.devices'])

# Show
df_d

,dms_timestamp,id,customer_fk,model_fk,serial_number,created,modified
0,2021-08-05 09:30:47,0,0,2,642-89-8554,1970-01-15 10:48:20,1970-01-15 11:08:12
1,2021-08-05 09:30:47,1,1,3,200-67-0041,2020-01-15 14:18:54,2020-01-15 14:54:32
2,2021-08-05 09:30:47,2,1,2,789-78-9293,2020-01-15 14:18:54,2020-01-15 14:43:00
3,2021-08-05 09:30:47,3,2,0,650-22-9565,2020-01-15 14:36:22,2020-01-15 15:18:43
4,2021-08-05 09:30:47,4,2,1,809-87-5733,2020-01-15 14:36:22,2020-01-15 15:03:44
5,2021-08-05 09:30:47,5,3,1,854-78-7933,2020-01-15 14:59:05,2020-01-15 15:48:19
6,2021-08-05 09:30:47,6,3,0,465-68-4500,2020-01-15 14:59:05,2020-01-15 15:17:12
7,2021-08-05 09:30:47,7,4,1,600-29-9728,2020-01-15 15:02:16,2020-01-15 15:09:16
8,2021-08-05 09:30:47,8,4,3,485-63-7817,2020-01-15 15:02:16,2020-01-15 15:14:06
9,2021-08-05 09:30:47,9,5,3,470-20-8539,2020-01-15 16:24:24,2020-01-15 16:56:35


### Customer_details

In [7]:
# Generate customer details. One per customer.
df_cd = generate_customer_details(df_c)

# Write
df_cd.to_parquet(table_paths['customers.customer_details'])

df_cd

,dms_timestamp,id,customer_fk,birthday,language,street_address,postal_code,city,country,phone_number,created,modified
0,2021-08-05 09:30:47,0,0,1915-02-12,ss,721 Hayes Ridge,21394,Herreraville,EC,698-615-5426x665,1970-01-15 10:00:00,1970-01-15 10:00:00
1,2021-08-05 09:30:47,1,1,1908-08-07,nr,081 Wyatt Harbors Apt. 671,35887,Hendersontown,MH,045.906.8991x685,2020-01-15 13:37:00,2020-01-15 13:37:00
2,2021-08-05 09:30:47,2,2,1946-03-10,lzh,17721 Jennifer Cape,13706,Nicholasberg,LV,(881)719-5610,2020-01-15 14:29:08,2020-01-15 14:29:08
3,2021-08-05 09:30:47,3,3,1985-03-26,vi,039 Jeff Estate,44287,Port Patrick,BY,001-240-687-7797x43638,2020-01-15 14:41:13,2020-01-15 14:41:13
4,2021-08-05 09:30:47,4,4,1910-03-19,kk,3765 Sarah Meadows,73010,West Meganberg,PT,038-600-1325,2020-01-15 14:58:59,2020-01-15 14:58:59
5,2021-08-05 09:30:47,5,5,1931-06-09,ce,54293 Wright Ports Apt. 612,47049,Kimchester,US,001-839-632-6952x5447,2020-01-15 15:36:56,2020-01-15 15:36:56
6,2021-08-05 09:30:47,6,6,1965-02-05,lv,4098 Lacey Mill Apt. 020,22406,Lake Thomasbury,BO,141-356-1342,2020-01-15 15:55:33,2020-01-15 15:55:33
7,2021-08-05 09:30:47,7,7,2017-11-26,kw,063 Steven Glens,32379,New Carolineview,BJ,+1-069-368-8980x8823,2020-01-15 16:17:08,2020-01-15 16:17:08
8,2021-08-05 09:30:47,8,8,1971-03-12,gd,195 Stephanie Stream Suite 284,37553,Moraleston,TG,001-402-842-2659x41091,2020-01-15 17:12:11,2020-01-15 17:12:11
9,2021-08-05 09:30:47,9,9,1980-04-21,bo,9383 Williams Path,60053,South Richardfort,BD,(949)958-5560,2020-01-15 17:36:47,2020-01-15 17:36:47


## List resulting files

In [13]:
# Maximum lenght of a path
max_w = max([len(x[1]) for x in table_paths.items()])

# UNIX epoch start
epoch = datetime(1970, 1, 1, tzinfo=timezone.utc)

for k, f in table_paths.items():

    s = os.stat(f)
    f_path = '.' + os.sep + f 
    f_size = s.st_size // 1024 
    f_mod = (
        (epoch + timedelta(seconds=s.st_mtime))
        .astimezone(tz.gettz('Europe/Helsinki'))
        .strftime('%y-%m-%d %H:%M:%S')
    )
    
    print(f"{f_mod} | {f_path:<{max_w}} | {f_size:>2d} KB")

21-08-05 09:30:47 | .\S3\staging\dms\abc\customers\customers\LOAD00000001.parquet      |  4 KB
21-08-05 09:30:47 | .\S3\staging\dms\abc\customers\customer_details\LOAD00000001.parquet |  9 KB
21-08-05 09:30:47 | .\S3\staging\dms\abc\devices\device_models\LOAD00000001.parquet    |  6 KB
21-08-05 09:30:47 | .\S3\staging\dms\abc\devices\devices\LOAD00000001.parquet          |  6 KB


## Preview only first lines

This is for screenshot purposes.

In [18]:
display(df_c.head(1))
display(df_dm.head(1))
display(df_d.head(1))
display(df_cd.head(1))

,dms_timestamp,id,username,created,modified
0,2021-08-05 09:30:47,0,janisourander@kamk.fi,1970-01-15 10:00:00,1970-02-20 12:34:56


,dms_timestamp,id,release_date,name,color,description,created,modified
0,2021-08-05 09:30:47,0,2010-05-15,Super Gadget 100,Red,lorem ipsum,2010-03-21 12:00:01,2010-03-21 12:00:01


,dms_timestamp,id,customer_fk,model_fk,serial_number,created,modified
0,2021-08-05 09:30:47,0,0,2,642-89-8554,1970-01-15 10:48:20,1970-01-15 11:08:12


,dms_timestamp,id,customer_fk,birthday,language,street_address,postal_code,city,country,phone_number,created,modified
0,2021-08-05 09:30:47,0,0,1915-02-12,ss,721 Hayes Ridge,21394,Herreraville,EC,698-615-5426x665,1970-01-15 10:00:00,1970-01-15 10:00:00
